# DSCI 100: Project Proposal 
### 1. Introduction

In [ ]:
# libraries 
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

### 2. Preliminary EDA

In [ ]:
audit <- read_csv("audit_data/audit_risk.csv") 
head(audit)

### 3. Methods

### 4. Expected Outcomes and Significance 